# Establish a Connection

In [1]:
# uncomment to install the packages
# restart the session after installing the packages

# ! pip install sentence-transformers==5.1.1 --upgrade
# ! pip install numpy==1.26.4 --upgrade

In [ ]:
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session

session = Session.builder.configs(SnowflakeLoginOptions("<your-connection>")).create()

SnowflakeLoginOptions() is deprecated since 1.8.5. 


In [3]:
from importlib.metadata import version

# batch inference PuPr in snowflake-ml-python>=1.26.0
print(version('snowflake-ml-python'))

1.26.0


# Create Resources

In [4]:
DB_NAME = "BATCH_INFERENCE_QUICKSTART_SENTENCE_TRANSFORMER_DB"
SCHEMA_NAME = "PUBLIC"
STAGE_NAME = "BATCH_INFERENCE_QUICKSTART_STAGE"
COMPUTE_POOL_NAME = "BATCH_INFERENCE_QUICKSTART_SENTENCE_TRANSFORMER_COMPUTE_POOL"

# Create database
session.sql(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}").collect()

# Create schema
session.sql(f"CREATE SCHEMA IF NOT EXISTS {DB_NAME}.{SCHEMA_NAME}").collect()

# Create stage with SSE encryption
session.sql(f"""
    CREATE STAGE IF NOT EXISTS {DB_NAME}.{SCHEMA_NAME}.{STAGE_NAME}
    ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE')
""").collect()

# Create compute pool with smallest CPU tier
# use GPU_NV_S for GPU workloads
session.sql(f"""
    CREATE COMPUTE POOL IF NOT EXISTS {COMPUTE_POOL_NAME}
    MIN_NODES = 1
    MAX_NODES = 2
    INSTANCE_FAMILY = CPU_X64_XS
""").collect()

# Set the session to use the newly created database and schema
session.use_database(DB_NAME)
session.use_schema(SCHEMA_NAME)

print(f"Created database: {DB_NAME}")
print(f"Created schema: {DB_NAME}.{SCHEMA_NAME}")
print(f"Created stage: {DB_NAME}.{SCHEMA_NAME}.{STAGE_NAME}")
print(f"Created compute pool: {COMPUTE_POOL_NAME}")

Created database: BATCH_INFERENCE_QUICKSTART_SENTENCE_TRANSFORMER_DB
Created schema: BATCH_INFERENCE_QUICKSTART_SENTENCE_TRANSFORMER_DB.PUBLIC
Created stage: BATCH_INFERENCE_QUICKSTART_SENTENCE_TRANSFORMER_DB.PUBLIC.BATCH_INFERENCE_QUICKSTART_STAGE
Created compute pool: BATCH_INFERENCE_QUICKSTART_SENTENCE_TRANSFORMER_COMPUTE_POOL


# Load and Log the SentenceTransformer Model

In [5]:
from sentence_transformers import SentenceTransformer
from snowflake.ml.registry import registry

input_data = [
    "This is the first sentence.",
    "Here's another sentence for testing.",
]

reg = registry.Registry(session=session, database_name=DB_NAME, schema_name=SCHEMA_NAME)

embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

mv = reg.log_model(
    embed_model,
    #version_name="V1",
    model_name="sentence_transformer_minilm",
    sample_input_data=input_data,
    pip_requirements=[
        "numpy==1.26.4",
        "sentence-transformers==5.1.1",
        "torch==2.9.0",
        "transformers==4.57.1",
    ],
)

/Users/hayu/miniconda3/envs/dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Logging model: validating model and dependencies...:   0%|          | 0/6 [00:00<?, ?it/s]

/Users/hayu/miniconda3/envs/dev/lib/python3.10/site-packages/snowflake/ml/registry/_manager/model_parameter_reconciler.py:155: UserWarning: Models logged specifying `pip_requirements` cannot be executed in a Snowflake Warehouse without specifying `artifact_repository_map`. This model can be run in Snowpark Container Services. See https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/container.
  warnings.warn(


Logging model: creating model manifest...:  33%|███▎      | 2/6 [00:01<00:03,  1.03it/s]  

/Users/hayu/miniconda3/envs/dev/lib/python3.10/site-packages/snowflake/ml/model/_packager/model_env/model_env.py:149: UserWarning: Dependencies specified from pip requirements. This may prevent model deploying to Snowflake Warehouse. Use 'artifact_repository_map' to deploy the model to Warehouse.
  self._warn_once(


Model logged successfully.: 100%|██████████| 6/6 [00:23<00:00,  3.99s/it]                          


# Create Input Dataset

In [6]:
import pandas as pd

# Define the data for the DataFrame
data = [
    ("The quick brown fox jumps over the lazy dog.", "a1b2c3d4-e5f6-7890-1234-567890abcdef"),
    ("Snowpark is a great library for data processing.", "f9e8d7c6-b5a4-3210-fedc-ba9876543210"),
    ("Python is a versatile programming language.", "1a2b3c4d-5e6f-7080-9101-112131415161")
]

# Define the column names and data types
columns = ["input_feature_0", "ID"]
schema = ["input_feature_0 VARCHAR", "ID VARCHAR(36)"]

# Create a pandas DataFrame
pandas_df = pd.DataFrame(data, columns=columns)

# Create the Snowpark DataFrame from the pandas DataFrame
snowpark_df = session.create_dataframe(pandas_df, schema=schema)

snowpark_df.show()

/Users/hayu/miniconda3/envs/dev/lib/python3.10/site-packages/snowflake/snowpark/_internal/utils.py:1148: UserWarning: data is a pandas DataFrame, parameter schema is ignored. To silence this warning pass schema=None.
  return func(*args, **kwargs)


-------------------------------------------------------------------------------------------
|"input_feature_0"                                 |"ID"                                  |
-------------------------------------------------------------------------------------------
|The quick brown fox jumps over the lazy dog.      |a1b2c3d4-e5f6-7890-1234-567890abcdef  |
|Snowpark is a great library for data processing.  |f9e8d7c6-b5a4-3210-fedc-ba9876543210  |
|Python is a versatile programming language.       |1a2b3c4d-5e6f-7080-9101-112131415161  |
-------------------------------------------------------------------------------------------



# Run the Batch Inference Job

In [7]:
from snowflake.ml.model import JobSpec, OutputSpec, SaveMode

output_stage_location = f"@{DB_NAME}.{SCHEMA_NAME}.{STAGE_NAME}/output/"

job = mv.run_batch(
    X=snowpark_df,
    compute_pool=COMPUTE_POOL_NAME,
    output_spec=OutputSpec(stage_location=output_stage_location, mode=SaveMode.OVERWRITE),
    job_spec=JobSpec(function_name="encode"),
    # job_spec=JobSpec(replicas=3, gpu_requests="1", num_workers=2) # uncomment it to try out custom job_spec
)

job.wait() # Wait for the job to complete

ModelVersion.run_batch() is in private preview since 1.18.0. Do not use it in production. 
/Users/hayu/miniconda3/envs/dev/lib/python3.10/site-packages/snowflake/ml/model/_client/model/model_version_impl.py:799: UserWarning: Output stage location '@BATCH_INFERENCE_QUICKSTART_SENTENCE_TRANSFORMER_DB.PUBLIC.BATCH_INFERENCE_QUICKSTART_STAGE/output/' is not empty. Found 2 existing files. OVERWRITE mode will remove all existing files in the output location before running the batch inference job.
  self._service_ops._enforce_save_mode(output_spec.mode, output_stage_location)


'DONE'

# Inspect the Inference Output

In [8]:
session.sql(f'LS {output_stage_location}').show()

----------------------------------------------------------------------------------------------------------------------------------
|"name"                                              |"size"  |"md5"                             |"last_modified"                |
----------------------------------------------------------------------------------------------------------------------------------
|batch_inference_quickstart_stage/output/3_07aa9...  |11627   |27033a315883bacf5e508df8af3a3daa  |Sat, 31 Jan 2026 00:15:06 GMT  |
|batch_inference_quickstart_stage/output/_SUCCESS    |0       |d41d8cd98f00b204e9800998ecf8427e  |Sat, 31 Jan 2026 00:15:06 GMT  |
----------------------------------------------------------------------------------------------------------------------------------



In [9]:
session.read.option("pattern", ".*\\.parquet").parquet(output_stage_location).show(1, max_width=200)

-------------------------------------------------------------------------------------------------------------------
|"input_feature_0"                             |"ID"                                  |"output_feature_0"         |
-------------------------------------------------------------------------------------------------------------------
|The quick brown fox jumps over the lazy dog.  |a1b2c3d4-e5f6-7890-1234-567890abcdef  |[                          |
|                                              |                                      |  4.393358901143074e-02,   |
|                                              |                                      |  5.893439799547195e-02,   |
|                                              |                                      |  4.817844927310944e-02,   |
|                                              |                                      |  7.754809409379959e-02,   |
|                                              |                        

## Copy the Output Stage Files into a Table

In [10]:
output_table = "batch_inference_output_table"
session.sql(f'CREATE OR REPLACE TABLE {DB_NAME}.{SCHEMA_NAME}.{output_table} (output VARIANT)').collect()
session.sql(f'COPY INTO {DB_NAME}.{SCHEMA_NAME}.{output_table} FROM {output_stage_location} FILE_FORMAT = (TYPE=parquet) ON_ERROR = CONTINUE').collect()
session.table(f'{DB_NAME}.{SCHEMA_NAME}.{output_table}').show()

------------------------------------------------------
|"OUTPUT"                                            |
------------------------------------------------------
|{                                                   |
|  "ID": "a1b2c3d4-e5f6-7890-1234-567890abcdef",     |
|  "input_feature_0": "The quick brown fox jumps...  |
|  "output_feature_0": [                             |
|    4.393358901143074e-02,                          |
|    5.893439799547195e-02,                          |
|    4.817844927310944e-02,                          |
|    7.754809409379959e-02,                          |
|    2.674440853297710e-02,                          |
|    -3.762958571314812e-02,                         |
|    -2.605181885883212e-03,                         |
|    -5.994307249784470e-02,                         |
|    -2.496046712622046e-03,                         |
|    2.207287400960922e-02,                          |
|    4.802591726183891e-02,                          |
|    5.575

# Clean Up

In [ ]:
# uncomment to clean up the database and compute pool
# session.sql(f'DROP DATABASE IF EXISTS {DB_NAME}').collect()
# session.sql(f'DROP COMPUTE POOL IF EXISTS {COMPUTE_POOL_NAME}').collect()

[Row(status='BATCH_INFERENCE_QUICKSTART_SENTENCE_TRANSFORMER_COMPUTE_POOL successfully dropped.')]